<a href="https://colab.research.google.com/github/lakshman111/.github.io/blob/master/virtualstagingcleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install transformers
!pip install diffusers
!pip install torchvision
!pip install accelerate
!pip install gradio
!pip install fastai
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import logging
from fastai.vision.all import *
import gradio as gr
import torch
from transformers import MaskFormerFeatureExtractor, MaskFormerForInstanceSegmentation
from diffusers import StableDiffusionInpaintPipeline, StableDiffusionDepth2ImgPipeline
from torchvision import transforms as tfms
from fastai.vision.all import *
import cv2
from PIL import Image as Image # image type to pass into the masking functions
from PIL import Image, ImageDraw # able to draw new images
#from IPython.display import Image as NotebookImage, display # image type for displaying in colab notebook

In [20]:
def pil_to_tensor(im):
    im = np.array(im) # To numpy array
    im = torch.tensor(im) # To tensor
    im = im.permute(2, 0, 1) # Rearrange dimensions
    im = im.unsqueeze(0) # Add batch dimension first
    return im/255.0 # Scale down to (0, 1)

def tensor_to_pil(tensor_im):
    # tensor_im = tensor_im.squeeze() # In case there is a batch dimension
    tensor_im = tensor_im.detach().cpu() # Detach from computational graph - explained in next lesson!
    tensor_im = tensor_im.permute(1, 2, 0) # Rearrange the channels
    tensor_im = tensor_im.clip(0, 1)*255 # Note that we clip to (0, 1) before scaling to (0, 255)
    im_array = np.array(tensor_im).astype(np.uint8) # Convert to int as required by PIL
    im = Image.fromarray(im_array) # Convert to PIL image
    return im

# Try to improve the mask thru convolutions etc
# assume m is a PIL object containing a grayscale 'diff'
def process_diffedit_mask(m,threshold=0.35,**kwargs):
    m = np.array(m).astype(np.float32)
    m = cv2.GaussianBlur(m,(5,5),1)
    m = (m>(255.*threshold)).astype(np.float32)*255
    m = Image.fromarray(m.astype(np.uint8))
    return m

def resize_pad_zeros(image, desired_size):
  tim = image.resize((desired_size, int(desired_size*(2/3))))
  return tim.crop_pad((512, 512), pad_mode="zeros")

In [21]:
# Set device
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

logger = logging.getLogger(__name__)

# Load Inpainting Model
inpaint = StableDiffusionInpaintPipeline.from_pretrained(
    'lakssrini/lvngrooms-room',
    torch_dtype=torch.float16
).to(torch_device)

# depth = StableDiffusionDepth2ImgPipeline.from_pretrained(
#     'lakssrini/dpt-lvngrooms',
#     torch_dtype=torch.float16
# ).to(torch_device)

# get the feature extractors
feature_extractor = MaskFormerFeatureExtractor.from_pretrained("facebook/maskformer-swin-large-ade")
model = MaskFormerForInstanceSegmentation.from_pretrained("facebook/maskformer-swin-large-ade")


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

In [41]:
# standard masks
#masks_to_ignore = ["wall", "ceiling", "window ", "door", "light", "curtain"]
masks_to_ignore = ["window ", "door", "light", "curtain"]

def make_mask(image):
    inputs = feature_extractor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    # model predicts class_queries_logits of shape `(batch_size, num_queries)`
    # and masks_queries_logits of shape `(batch_size, num_queries, height, width)`
    class_queries_logits = outputs.class_queries_logits
    masks_queries_logits = outputs.masks_queries_logits

    pred_scores, pred_labels = F.softmax(class_queries_logits, dim=-1).max(-1)

    count = 0
    feature_index_dict = {}
    new_logits = masks_queries_logits.squeeze(0).detach()
    for score, label in zip(pred_scores.squeeze(0), pred_labels.squeeze(0)):
        if score < 0.5 or label.item() == 150:
            count += 1
            continue
        mask = new_logits[count]
        label_name = model.config.id2label[label.item()]
        feature_index_dict[label_name] = (mask, score)
        count += 1

    seg_final = None
    
    for key, mask_tuple in feature_index_dict.items():
        if key not in masks_to_ignore:
            logger.info(f"adding a {key}: score: {mask_tuple[1]}, count: {count}")
            if seg_final is None:
                seg_final = torch.sigmoid(mask_tuple[0])
            else:
                seg_final += torch.sigmoid(mask_tuple[0])
            
    
    seg_final = tfms.GaussianBlur((5,5), 1)(torch.cat(3*[seg_final.unsqueeze(0)]))
    mask_temp = tensor_to_pil((seg_final>0.35).float()).resize(image.size)

    #NEW CODE FROM LAKSH

    #save the mask_temp as a file
    mask_temp.save("generated_mask.png")

    # Create a new image with a mode of RGBA and the specified size
    default_split_mask = Image.new("RGBA", (512, 512))

    # Get the draw object
    draw = ImageDraw.Draw(default_split_mask)

    # Draw a black rectangle on the top half of the image
    draw.rectangle([(0, 0), (512, 256)], fill=(0, 0, 0, 255))

    # Save the image as a png file
    default_split_mask.save("default_split_mask.png")
    #display(default_split_mask)

    #access the generated mask
    generated_mask = Image.open("generated_mask.png").convert("RGBA")

    # Overlay the top image on top of the bottom image
    final_mask = Image.alpha_composite(generated_mask, default_split_mask)

    # Save the result
    final_mask.save("final_mask.png")

    return final_mask

# removed depth model
# def make_new_style(image, use_depth_model, prompt, neg_prompt, guidance_scale, seed=0):
def make_new_style(image, prompt, email):
    
    #email
    with open("database.txt", "a") as file:
      file.write("\n"+email)

    if prompt == "Modern":
      prompt = "a modern living room"

    neg_prompt = "Oversaturated, blurry, low quality"

    guidance_scale = 9

    seed = -1

    # fix bug where image is inputted on mobile vertically
    
    # # Load the image
    # image.save("new_image.png")
    # img = Image.open("new_image.png")

    # # Convert PIL Image to OpenCV image
    # img = np.array(img)

    # # Get the orientation of the image
    # rows,cols = img.shape[:2]
    # print(f"rows: {rows}, cols: {cols}")

    # if rows > cols:
    #     orientation = "portrait"
    # else:
    #     orientation = "landscape"

    # # Rotate the image
    # if orientation == "portrait":
    #   img = cv2.transpose(img)
    #   img = cv2.flip(img,1)

    # # Save the rotated back to the original
    # image = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # image = image.resize((512, 512))
    
    generator = torch.Generator(torch_device)
    generator = generator.manual_seed(seed) if seed > 0 else None
    # image = image.resize((640, 480))
    im_result = []

    # removing depth model functionality for now
    use_depth_model = False
    if use_depth_model:
        pass
        # im_result.extend(
        #     depth(
        #         prompt=f"{prompt.strip()} like lvngrooms room",
        #         image=image,
        #         num_images_per_prompt=2,
        #         negative_prompt=neg_prompt,
        #         guidance_scale=guidance_scale,
        #         generator=generator,
        #         num_inference_steps=45,
        #         strength=0.9
        #     ).images
        # ) 
    else:
        mask = make_mask(image)
        im_result.extend(
            inpaint(
                prompt=f"{prompt.strip()} like lvngrooms room",
                image=image,
                mask_image=mask,
                num_images_per_prompt=2,
                negative_prompt=neg_prompt,
                guidance_scale=guidance_scale,
                generator=generator,
                num_inference_steps=50,
                # height=480,
                # width=640
                height=512,
                width=512,
                # strength=0.85
            ).images
        ) 
    
    return im_result


In [48]:
demo = gr.Interface(
    fn=make_new_style,
    allow_flagging="never",
    #can switch to  allow_flagging="auto", flagging_callback=hf_writer): https://www.gradio.app/docs/#huggingfacedatasetsaver
    inputs=[ 
        gr.components.Image(label="Desktop only (no mobile support). File type: JPG, JPEG, and PNG",type="pil", shape=(512, 512)),
        #gr.components.Checkbox(label="Use depth model", value=False),
        gr.components.Dropdown(["Modern"],value = "Modern",label="Style"),
        #gr.components.Textbox(label="Negative Prompt", placeholder="Oversaturated, blurry, low quality"),
        #gr.components.Slider(minimum=7, maximum=12, value=9),
        gr.components.Textbox(label="Email (required)", placeholder="example@email.com"),
        #gr.components.Number(value=944895745, precision=0) 
    ], 
    outputs=[
        gr.components.Image(type="pil", shape=(512, 512), label="output-1"),
        gr.components.Image(type="pil", shape=(512, 512), label="output-2"),
    ], 
)

demo.launch(debug=True, share=True)  

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://fbbf9039-9f38-4fcf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://fbbf9039-9f38-4fcf.gradio.live
